In [53]:
import pickle
from nltk import sent_tokenize, word_tokenize

In [14]:
import re
def extract_features(sentence, index):
  return {
      'word':sentence[index],
      'is_first':index==0,
      'is_last':index ==len(sentence)-1,
      'prefix-1':sentence[index][0],
      'prefix-2':sentence[index][:2],
      'prefix-3':sentence[index][:3],
      'prefix-3':sentence[index][:4],
      'suffix-1':sentence[index][-1],
      'suffix-2':sentence[index][-2:],
      'suffix-3':sentence[index][-3:],
      'suffix-3':sentence[index][-4:],
      'prev_word':'' if index == 0 else sentence[index-1],
      'next_word':'' if index < len(sentence) else sentence[index+1],
      'has_hyphen': '-' in sentence[index],
      'is_numeric': sentence[index].isdigit(),
  }

In [7]:
with open("./pos_crf_model.pkl", "rb") as f:
    model = pickle.load(f)
    print(type(model))

<class 'sklearn_crfsuite.estimator.CRF'>


In [52]:
def transform_to_dataset(sentence):
  tmp = []
  for index in range(len(sentence)):
      tmp.append(extract_features(sentence, index)),
  return tmp

In [57]:
def transform_to_dataset(corpus: str):
    sentences = sent_tokenize(corpus)
    sentences = [word_tokenize(sentence) for sentence in sentences]
    X = []
    for sentence in sentences:
        tmp = []
        for i in range(len(sentence)):
            tmp.append(extract_features(sentence, i))
        X.append(tmp)
    return X

In [58]:
input_txt = "இல்­லையா என்­பதை மக்கள் முன்­னி­லையில் குறிப்­பிட வேண்டும் .  அதேபோல் இம்­முறை வரவு–செலவு திட்டமானது மக்­க­ளுக்கு சிறி­த­ளவு நிவா­ரணம் அம்­சங்கள் சிலவும் உள்­ளன .  அதனால் இதனை தோற்கடிப்பதால் மாத்திரம் மாற்றத்தை ஏற்படுத்த முடியாது."

In [60]:
X = transform_to_dataset(input_txt)

In [47]:
X = [transform_to_dataset(input_txt)]

In [48]:
type(X)

list

In [61]:
print(X[0])

[{'word': 'இல்\xadலையா', 'is_first': True, 'is_last': False, 'prefix-1': 'இ', 'prefix-2': 'இல', 'prefix-3': 'இல்\xad', 'suffix-1': 'ா', 'suffix-2': 'யா', 'suffix-3': 'லையா', 'prev_word': '', 'next_word': '', 'has_hyphen': False, 'is_numeric': False}, {'word': 'என்\xadபதை', 'is_first': False, 'is_last': False, 'prefix-1': 'எ', 'prefix-2': 'என', 'prefix-3': 'என்\xad', 'suffix-1': 'ை', 'suffix-2': 'தை', 'suffix-3': '\xadபதை', 'prev_word': 'இல்\xadலையா', 'next_word': '', 'has_hyphen': False, 'is_numeric': False}, {'word': 'மக்கள்', 'is_first': False, 'is_last': False, 'prefix-1': 'ம', 'prefix-2': 'மக', 'prefix-3': 'மக்க', 'suffix-1': '்', 'suffix-2': 'ள்', 'suffix-3': '்கள்', 'prev_word': 'என்\xadபதை', 'next_word': '', 'has_hyphen': False, 'is_numeric': False}, {'word': 'முன்\xadனி\xadலையில்', 'is_first': False, 'is_last': False, 'prefix-1': 'ம', 'prefix-2': 'மு', 'prefix-3': 'முன்', 'suffix-1': '்', 'suffix-2': 'ல்', 'suffix-3': 'யில்', 'prev_word': 'மக்கள்', 'next_word': '', 'has_hyphen'

In [62]:
model.predict(X)

[['NOUN', 'VERB', 'NOUN', 'NOUN', 'VERB', 'VAUX', 'PUNCT'],
 ['PRON',
  'NOUN',
  'NOUN',
  'NOUN',
  'NOUN',
  'NOUN',
  'NOUN',
  'NOUN',
  'NOUN',
  'VERB',
  'PUNCT'],
 ['CCONJ', 'PRON', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'VAUX', 'PUNCT']]

In [24]:
extract_features(["இல்­லையா"], 0)

{'word': 'இல்\xadலையா',
 'is_first': True,
 'is_last': True,
 'prefix-1': 'இ',
 'prefix-2': 'இல',
 'prefix-3': 'இல்\xad',
 'suffix-1': 'ா',
 'suffix-2': 'யா',
 'suffix-3': 'லையா',
 'prev_word': '',
 'next_word': '',
 'has_hyphen': False,
 'is_numeric': False}

----

In [63]:
with open("./ner_crf_model.pkl", "rb") as f:
    ner_model = pickle.load(f)

In [64]:
ner_model.classes_

['O', 'B-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'I-LOC', ' B-ORG']

In [65]:
def extract_ner_features(sentence, index):
    word = sentence[index]
    return {
        'word': word,
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'prefix-1': "" if not word else word[0],
        'prefix-2': word[:2],
        'prefix-3': word[:3],
        'prefix-4': word[:4],
        'suffix-1': "" if not word else word[-1],
        'suffix-2': word[-2:],
        'suffix-3': word[-3:],
        'suffix-4': word[-4:],
        'prev_word': '' if index == 0 else sentence[index - 1][0],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1][0],
        'has_hyphen': '-' in word,
        'is_numeric': word.isdigit(),
    }

In [66]:
def transform_to_ner_dataset(corpus: str):
    sentences = sent_tokenize(corpus)
    sentences = [word_tokenize(sentence) for sentence in sentences]
    X = []
    for sentence in sentences:
        tmp = []
        for i in range(len(sentence)):
            tmp.append(extract_ner_features(sentence, i))
        X.append(tmp)
    return X

In [67]:
input_txt

'இல்\xadலையா என்\xadபதை மக்கள் முன்\xadனி\xadலையில் குறிப்\xadபிட வேண்டும் .  அதேபோல் இம்\xadமுறை வரவு–செலவு திட்டமானது மக்\xadக\xadளுக்கு சிறி\xadத\xadளவு நிவா\xadரணம் அம்\xadசங்கள் சிலவும் உள்\xadளன .  அதனால் இதனை தோற்கடிப்பதால் மாத்திரம் மாற்றத்தை ஏற்படுத்த முடியாது.'

In [69]:
X = transform_to_ner_dataset(input_txt)

In [71]:
ner_model.predict(X)

[['O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]